# Bring the stock data using library

In [19]:
#pip install pandas-datareader
#!pip install py7zr
#!pip install newspaper3k
#!pip install yfinance
#!pip install torch
#!pip install yfinance
import torch
import yfinance as yf
import py7zr
import numpy as np
import pandas as pd
from newspaper import Article
import pandas_datareader.data as reader
import os
import time
import re
import warnings

import yfinance as yf

warnings.filterwarnings("ignore")

In [ ]:

from pandas_datareader.nasdaq_trader import get_nasdaq_symbols
nasdaq_list = get_nasdaq_symbols()
nasdaq_list.head()

In [20]:
#import data from a certain period, CERTAIN COMPANY
ticker = 'AAPL'
start_date = '2021-01-01'
end_date = '2021-12-31'

df = yf.download(ticker, start=start_date, end=end_date)
print(df.head())

[*********************100%***********************]  1 of 1 completed

Price            Close        High         Low        Open     Volume
Ticker            AAPL        AAPL        AAPL        AAPL       AAPL
Date                                                                 
2021-01-04  126.544205  130.651192  123.952888  130.563189  143301900
2021-01-05  128.108765  128.822609  125.585897  126.035717   97664900
2021-01-06  123.796448  128.147907  123.581318  124.891648  155088000
2021-01-07  128.020782  128.715066  125.028548  125.517476  109578200
2021-01-08  129.125763  129.692921  127.346060  129.497338  105158200


Since I can't seem to find a separate list for NASDAQ medical/bio stocks,
I will go right into text crawling to find out which companies are medical/bio stocks.

# Text Crawling



In [ ]:
# 경로 설정
PATH = "your/directory/path"
READ_PATH = os.path.join(PATH, "input")
WRITE_PATH = os.path.join(PATH, "output")

# 데이터 로드
nasdaq_df = pd.read_csv(os.path.join(READ_PATH, 'nasdaq_rss_ifo.csv'))

# 1. 뉴스 데이터 전처리 및 크롤링
def extract_text(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        return article.text
    except Exception as e:
        print(f"Error processing URL {url}: {e}")
        return None

nasdaq_df['text'] = nasdaq_df['url_ifo'].apply(extract_text)
nasdaq_df.dropna(subset=['text'], inplace=True)

# 크롤링 데이터 저장
nasdaq_df.to_csv(os.path.join(WRITE_PATH, 'nasdaq_with_text.csv'), index=False)

# 2. 기업 설명 크롤링
stock = pd.read_csv(os.path.join(READ_PATH, 'NASDAQ_FC_STK_IEM_IFO.csv'), encoding="cp949")

def get_stock_descriptions(ticker_list):
    descriptions = []
    for ticker in ticker_list:
        try:
            stock_info = yf.Ticker(ticker).info
            descriptions.append(stock_info.get('longBusinessSummary', None))
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")
            descriptions.append(None)
    return pd.DataFrame({'tck_iem_cd': ticker_list, 'description': descriptions})

# 종목 설명 데이터 생성
tck_iem_cds = stock['tck_iem_cd'].str.strip()
stock_description = get_stock_descriptions(tck_iem_cds)
stock_description.dropna(inplace=True)

# 데이터 저장
stock_description.to_csv(os.path.join(WRITE_PATH, 'stock_descriptions.csv'), index=False)

# 3. AI 및 의료 관련 주식 필터링
# AI 및 의료 관련 키워드
keywords = ['AI', 'artificial intelligence', 'medical', 'biotech', 'healthcare']

# 뉴스 텍스트와 기업 설명 데이터에서 키워드 검색
nasdaq_df['related_to_ai_medical'] = nasdaq_df['text'].apply(
    lambda x: any(keyword.lower() in x.lower() for keyword in keywords)
)
stock_description['related_to_ai_medical'] = stock_description['description'].apply(
    lambda x: any(keyword.lower() in x.lower() for keyword in keywords)
)

# 관련 주식 식별
related_news_stocks = nasdaq_df[nasdaq_df['related_to_ai_medical']]['tck_iem_cd'].unique()
related_description_stocks = stock_description[stock_description['related_to_ai_medical']]['tck_iem_cd'].unique()

# 교집합 및 데이터 정리
related_stocks = set(related_news_stocks).union(set(related_description_stocks))
related_stocks_df = stock[stock['tck_iem_cd'].isin(related_stocks)]

# 결과 저장
related_stocks_df.to_csv(os.path.join(WRITE_PATH, 'related_ai_medical_stocks.csv'), index=False)
